In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
from statsmodels.stats.api import durbin_watson, acorr_breusch_godfrey, cov_hac # DW & LM-тесты
from statsmodels.iolib.summary2 import summary_col # вывод подгонки

from scipy.stats import chi2 # chi2-распределение и критические значения

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warning
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Диагностика модели: тесты на серийную корреляцию</span> </b><br>

Рассмотрим __линейную регрессию для временных рядов__ с серийной корреляцией для ошибки (внешнего шока)

$$
\begin{aligned}
	y_t&=x_t^\top\beta+u_t \\ u_t&=\rho_1u_{t-1}+\cdots+\rho_pu_{t-p}+v_t
\end{aligned}
$$
где
* $y_t,x_t$ наблюдаемы
* $u_t$ - эндогенный внешний шок (__ненаблюдаем!__)
* $v_t$ - экзогенный внешний шок (__ненаблюдаем!__)
* $p$ - порядок серийной корреляции/автокорреляции

**<span style="color:purple">DW-тест:</span>** тестируем автокорреляцию __первого порядка__, т.е. для модели

$$
\begin{aligned}
	y_t&=x_t^\top\beta+u_t \\ u_t&=\rho u_{t-1}+v_t
\end{aligned}
$$
тестируем

$$
\begin{aligned}
	H_0:\rho&=0 & &vs & H_1:\rho&\ne0
\end{aligned}
$$

__Тестовая статистика__

$$
\begin{aligned}
	DW&=\frac{\sum_{t=2}^n(e_t-e_{t-1})^2}{\sum_{t=1}^n e^2_t} & 0\leq DW&\leq4
\end{aligned}
$$

__Критические значения__: из специальной таблицы находим два критических значения $0<d_\ell<d_u$ (зависят от $n,k,\alpha$)

__Вывод:__ 

|Значение $DW$ |Гипотеза $H_0$|Вывод |
|:--:|:--:|:--:|
|$0 \leq DW< d_L$|отвергается|есть положительная автокорреляция|
|$d_L \leq DW \leq d_U$ |?|неопределенность |
|$d_U < DW < 4-d_U$ |не отвергается |автокорреляция отсутствует |
|$4-d_U \leq DW \leq 4-d_L$|?|неопределенность |
|$4-d_L < DW \leq 4$ |отвергается|есть отрицательная автокорреляция|

__Вывод (приближённый):__
* Не отвергаем гипотезу при $DW\approx2$ (нет серийной корреляции)
* Отвергаем гипотезу если $DW$ "сильно отличается от 2" (есть серийная корреляция)

__Замечание:__ 

* используем метод [`statsmodels.stats.stattools.durbin_watson(resids)`](https://www.statsmodels.org/stable/generated/statsmodels.stats.stattools.durbin_watson.html#statsmodels.stats.stattools.durbin_watson)
* где `resids` -  остатки модели
* критические значения берём из специальной таблицы
 

**<span style="color:purple">LM/BG-тест:</span>** тест на  серийную корреляцию произвольного порядка

$$
	H_0:\rho_1=\cdots=\rho_p=0
$$

Тест основан на вспомогательной (неинтерпретируемой!) регрессии для OLS-остатков

$$
\begin{aligned}
	e_t\;&\text{на}\; e_{t-1},\ldots,e_{t-p}, x & &R^2_{aux}
\end{aligned}
$$

__Два подхода к тестированию__
* тестовая статистика $LM=(n-p)R^2_{aux}$, критическое значение $\chi^2_{cr}=\chi^2_{df=p}(\alpha)$ ($p$ - порядок серийной корреляции)
* F-тест на совместную значимость $e_{t-1},\ldots,e_{t-p}$

__Вывод:__ Тест указывает на серийную корреляцию (отвергаем $H_0$) при $LM>\chi^2_{cr}, F>F_{cr},P<\alpha$. 

__Замечание:__ 

* используем метод [`statsmodels.stats.diagnostic.acorr_breusch_godfrey(res, nlags)`](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.acorr_breusch_godfrey.html#statsmodels.stats.diagnostic.acorr_breusch_godfrey)
* где `res` -  подогнанная модель
* `nlags` – порядок автокорреляции
* методы возвращает четыре числа: $LM$-статистику и её P-значение, $F$-статистику и её P-значение

</div>

спец-таблицы н.12-13, k-количество регрессоров, dl и dU - нижняя и верхняя границы интервала

In [4]:
# загрузим данные
df = pd.read_csv('Mishkin.csv')
df

,rownames,pai1,pai3,tb1,tb3,cpi
0,1,-3.552289,1.129370,1.100854,1.129406,23.5
1,2,5.247540,4.001566,1.125513,1.137254,23.6
2,3,1.692860,4.492160,1.115715,1.142319,23.6
3,4,5.064298,7.817513,1.146380,1.177902,23.7
4,5,6.719322,9.433580,1.158520,1.167777,23.8
...,...,...,...,...,...,...
486,487,10.992540,9.397943,7.468623,7.662047,131.6
487,488,9.988676,6.631662,7.438210,7.548033,132.7
488,489,7.212614,3.302103,7.026882,7.293367,133.5
489,490,2.693695,3.282396,6.626972,7.264110,133.8


In [5]:
# т.к. месячные данные, то зададим периодический временной индекс с 1950-02 по 1990-12
# параметр freq задаёт периодичность
df.index = pd.period_range(start='1950-02', end='1990-12', freq='M')
df

,rownames,pai1,pai3,tb1,tb3,cpi
1950-02,1,-3.552289,1.129370,1.100854,1.129406,23.5
1950-03,2,5.247540,4.001566,1.125513,1.137254,23.6
1950-04,3,1.692860,4.492160,1.115715,1.142319,23.6
1950-05,4,5.064298,7.817513,1.146380,1.177902,23.7
1950-06,5,6.719322,9.433580,1.158520,1.167777,23.8
...,...,...,...,...,...,...
1990-08,487,10.992540,9.397943,7.468623,7.662047,131.6
1990-09,488,9.988676,6.631662,7.438210,7.548033,132.7
1990-10,489,7.212614,3.302103,7.026882,7.293367,133.5
1990-11,490,2.693695,3.282396,6.626972,7.264110,133.8


## Спецификация и подгонка

__Замечание__: сначала подготовим датасет, т.е. выполним необходимые преобразования данных и добавим новые переменные:
* дифференцируем `pai3, tb3` (новые переменные `diff_pai3, diff_tb3`)
* log-дифференцируем `cpi` (новая переменная `diff_log_cpi`)

In [6]:
# добавим новые переменные в датасет:
df[ ['diff_pai3', 'diff_tb3'] ] = df[ ['pai3', 'tb3'] ].diff()
df['diff_log_cpi'] = np.log(df['cpi']).diff()
df

,rownames,pai1,pai3,tb1,tb3,cpi,diff_pai3,diff_tb3,diff_log_cpi
1950-02,1,-3.552289,1.129370,1.100854,1.129406,23.5,NaN,NaN,NaN
1950-03,2,5.247540,4.001566,1.125513,1.137254,23.6,2.872196,0.007848,0.004246
1950-04,3,1.692860,4.492160,1.115715,1.142319,23.6,0.490594,0.005065,0.000000
1950-05,4,5.064298,7.817513,1.146380,1.177902,23.7,3.325353,0.035583,0.004228
1950-06,5,6.719322,9.433580,1.158520,1.167777,23.8,1.616067,-0.010125,0.004211
...,...,...,...,...,...,...,...,...,...
1990-08,487,10.992540,9.397943,7.468623,7.662047,131.6,0.867525,-0.251260,0.009160
1990-09,488,9.988676,6.631662,7.438210,7.548033,132.7,-2.766281,-0.114014,0.008324
1990-10,489,7.212614,3.302103,7.026882,7.293367,133.5,-3.329559,-0.254666,0.006011
1990-11,490,2.693695,3.282396,6.626972,7.264110,133.8,-0.019707,-0.029257,0.002245


In [8]:
# спецификация модели через формулу
mod = smf.ols(formula='diff_pai3~1+diff_tb3+diff_log_cpi', data=df)
# подгонка модели с ковариационной матрицей по умолчанию (неробастной)
res = mod.fit()

In [9]:
# DW-статистика
durbin_watson(res.resid)

1.6211050478667361

In [11]:
#число наблюдений, по которым строилась модель
res.nobs

490.0

In [12]:
#1.62 сильно отличается от 2, а 1.91 не сильно....

diff(lhur) ~ 1 + diff_log_punew + diff_fyff + diff_fygm3 + diff(fygt1) + diff(log(gdpjp))